In [45]:
import pyspark
import requests
import datetime

from pyspark.sql import SparkSession 

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()




In [50]:
noise = 99
hashed_column = 'surname'

start = datetime.datetime.now()

request = requests.get(f'http://cluster-a:9200//take_data/{hashed_column}/{noise}')
url_content = request.content
csv_file = open("/opt/workspace/a_download.csv", 'wb')
csv_file.write(url_content)
csv_file.close()

request = requests.get(f'http://cluster-b:9300//take_data/{hashed_column}/{noise}')
url_content = request.content
csv_file = open("/opt/workspace/b_download.csv", 'wb')
csv_file.write(url_content)
csv_file.close()


df_1 = spark.read.csv(path="/opt/workspace/a_download.csv", sep=",", header=True)
df_2 = spark.read.csv(path="/opt/workspace/b_download.csv", sep=",", header=True)

df_1.createOrReplaceTempView("a_cluster_data")
df_2.createOrReplaceTempView("b_cluster_data")

sql_result = spark.sql("  SELECT DISTINCT a_cluster_data.id_a, b_cluster_data.id_b, a_cluster_data.surname as hashed_data\
                          FROM a_cluster_data, b_cluster_data\
                          WHERE a_cluster_data.surname == b_cluster_data.surname\
                          ORDER BY hashed_data ASC")

sql_result.select('id_a').drop_duplicates().repartition(1).write.mode('overwrite').csv("/opt/workspace//id_df_a")
sql_result.select('id_b').drop_duplicates().repartition(1).write.mode('overwrite').csv("/opt/workspace//id_df_b")

stop = datetime.datetime.now()

print(stop - start)

0:00:04.729178


In [62]:
df_2.count()

1094

0:00:00.414991


In [44]:
myfiles = {'file': open('/opt/workspace/id_df_a/part-00000-2cc8bcdd-f1bf-4411-9a7d-50b32be8788b-c000.csv' ,'rb')}
requests.post(f'http://cluster-a:9200//take_data/', data={'column': hashed_column}, files = myfiles)

myfiles = {'file': open('/opt/workspace/id_df_b/part-00000-eb201cb5-f83a-4019-8c79-014226656171-c000.csv' ,'rb')}
requests.post(f'http://cluster-b:9300//take_data/', data={'column': hashed_column}, files = myfiles)

FileNotFoundError: [Errno 2] No such file or directory: '/opt/workspace/id_df_a/part*.csv'

In [42]:
result_1 = requests.get(f'http://cluster-b:9300//send_data_to_alice')
print(result_1.content)


b'Everything was completed succesfully'


In [43]:

result_2 = requests.get(f'http://cluster-a:9200//send_data_to_bob')
print(result_2.content)

b'Everything was completed succesfully'
